In [1]:
import os
import pandas as pd

import requests
from bs4 import BeautifulSoup

import re

path = os.path.abspath(os.path.dirname(os.getcwd())) + '\\data'

In [2]:
# part 1 : crawling the website for each issue

issue_df = pd.read_csv(path+'\\website\\issue_new.csv')

def find_website_article(i):
    
    number_volume = issue_df["number_volume"].iloc[i]
    number_issue = issue_df["number_issue"].iloc[i]
    
    url_issue = issue_df["website_issue"].iloc[i]
    response = requests.get(url=url_issue)
    
    web_content = response.content
    bs_obj=BeautifulSoup(web_content,'lxml').body.div.div
    
    article = bs_obj.find_all(name='div', attrs={"class":"articleEntry"})
    
    article_df = pd.DataFrame(columns=["number_volume","number_issue","number_article","type","link"])
    
    for i in range(len(article)):
        try:
            type = article[i].find(name='div', attrs={"class":"article-type"}).string
        except:
            type = ""
            
        link_str = str(article[i].find(name='div', attrs={"class":"art_title linkable"}))
        link = re.findall(r"href=\"(.+?)\">", link_str)[0]
        link = 'https://www.tandfonline.com' + link
        
        article_df.loc[i] = [number_volume, number_issue, i+1, type, link]
    
    return article_df

In [3]:
num_issue = issue_df.shape[0]

for i_issue in range(num_issue):
    
    article_df = find_website_article(i_issue)
    
    number_volume = article_df["number_volume"].iloc[0]
    number_issue = article_df["number_issue"].iloc[0]
    
    article_df.to_csv(path+'\\website_article\\article_vol'+str(number_volume)+
                      '_isu'+str(number_issue)+'.csv',index=False,sep=',')

In [12]:
# part 2 : put all the webistes into one form

issue_df = pd.read_csv(path+'\\website\\issue_new.csv')
iter_issue = zip(issue_df['number_volume'],issue_df['number_issue'])

article_df_all = pd.DataFrame(columns=["number_volume","number_issue",
                                       "number_article","type","link"])

for number_volume,number_issue in iter_issue:
    
    article_df = pd.read_csv(path+'\\website_article\\article_vol'+str(number_volume)+
                             '_isu'+str(number_issue)+'.csv')
    
    article_df_all = pd.concat([article_df_all, article_df], axis=0, ignore_index=True)


article_df_all.rename(columns={'type':'type_1'}, inplace=True)
article_df_all.tail()

,number_volume,number_issue,number_article,type_1,link
912,23,2,2,Article,https://www.tandfonline.com/doi/full/10.1080/1...
913,23,2,3,Article,https://www.tandfonline.com/doi/full/10.1080/1...
914,23,2,4,Article,https://www.tandfonline.com/doi/full/10.1080/1...
915,23,2,5,Article,https://www.tandfonline.com/doi/full/10.1080/1...
916,23,2,6,book review,https://www.tandfonline.com/doi/full/10.1080/1...


In [13]:
article_df_all.to_csv(path+'\\website_article\\article.csv',index=False,sep=',')